Fechamento do BOOK de operações.

In [1]:
import ccxt
import pandas as pd
import datetime as dt
# import numpy as np
import config
import Mempoolpy
import Ordipy

In [2]:
def start_exchange(exchange_id, contract_type):

    if (exchange_id == 'binanceusdm') or (exchange_id == 'binance'):
        api_key = config.api_key_binance
        api_secret = config.api_secret_binance

    elif exchange_id == 'mexc':
        api_key = config.api_key_mexc
        api_secret = config.api_secret_mexc

    elif exchange_id == 'bybit':
        api_key = config.api_key_bybit
        api_secret = config.api_secret_bybit

    elif exchange_id == 'gate':
        api_key = config.api_key_gate
        api_secret = config.api_secret_gate

    elif exchange_id == 'binance_sub':
        exchange_id = 'binance'
        api_key = config.api_key_binance_sub
        api_secret = config.api_secret_binance_sub

    elif exchange_id == 'binanceusdm_sub':
        exchange_id = 'binanceusdm'
        api_key = config.api_key_binance_sub
        api_secret = config.api_secret_binance_sub

    options = {}
    if contract_type == 'swap':
        options['defaultType'] = 'swap'
    else:
        options['defaultType'] = ''
    exchange_class = getattr(ccxt, exchange_id)
    exchange = exchange_class({
        'apiKey': api_key,
        'secret': api_secret,
        'options': options if options else None
    })
    # 'options': {**options, 'adjustForTimeDifference': True, 'recvWindow': 5000} if options else {'adjustForTimeDifference': True}
    exchange.check_required_credentials()
    return exchange


def create_balance_exchange(exc, exchange, exchange_id):
    if exc == 'mexc':
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange = balanceNow[balanceNow['total'] != 0].copy()

        exchange = start_exchange(exc, contract_type='swap')
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange2 = balanceNow[balanceNow['total'] != 0].copy()
        balance_exchange = pd.concat([balance_exchange, balance_exchange2])

    elif exc == 'gate':
        balance = pd.DataFrame(exchange.fetchBalance(params={})['info'])
        balance_exchange = balance.drop(columns=['update_id']).rename(columns={'currency': '', 'available': 'free', 'locked': 'used'})
        balance_exchange.index = balance_exchange['']
        balance_exchange = balance_exchange.drop(columns=[''])
        balance_exchange['total'] = balance_exchange['free'].astype(float) + balance_exchange['used'].astype(float)
        balance_exchange = balance_exchange[balance_exchange['total'] != 0]

    else:
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange = balanceNow[balanceNow['total'] != 0].copy()
        if exc == exchange_id[0]:
            balance_exchange.reset_index(inplace=True)
            balance_exchange['index'] = balance_exchange['index'].str.replace('LD','')
            balance_exchange.columns = ['', 'free', 'used', 'total']
            balance_exchange = balance_exchange.groupby([''])[['free', 'used', 'total']].sum()
    return balance_exchange




start

In [3]:
exchange_id = ['binance', 'binanceusdm', 'mexc', 'bybit', 'binance_sub', 'binanceusdm_sub']
exchange_id = ['mexc']
balance_exchange_save = pd.DataFrame()
date_time = dt.datetime.now()

for exc in exchange_id:
    print(exc)
    exchange = start_exchange(exc, '')
    if exc == 'mexc':
        exchange.enableRateLimit = True
        exchange.options['adjustForTimeDifference'] = True
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange = balanceNow[balanceNow['total'] != 0].copy()

        exchange = start_exchange(exc, contract_type='swap')
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange2 = balanceNow[balanceNow['total'] != 0].copy()
        balance_exchange = pd.concat([balance_exchange, balance_exchange2])

    elif exc == 'gate':
        balance = pd.DataFrame(exchange.fetchBalance(params={})['info'])
        balance_exchange = balance.drop(columns=['update_id']).rename(columns={'currency': '', 'available': 'free', 'locked': 'used'})
        balance_exchange.index = balance_exchange['']
        balance_exchange = balance_exchange.drop(columns=[''])
        balance_exchange['total'] = balance_exchange['free'].astype(float) + balance_exchange['used'].astype(float)
        balance_exchange = balance_exchange[balance_exchange['total'] != 0]

    else:
        balance = pd.DataFrame(exchange.fetchBalance(params={}))
        balance_to_pivot = balance.drop(columns=['info'])
        balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
        balance_exchange = balanceNow[balanceNow['total'] != 0].copy()
        if exc == exchange_id[0]:
            balance_exchange.reset_index(inplace=True)
            balance_exchange['index'] = balance_exchange['index'].str.replace('LD','')
            balance_exchange.columns = ['', 'free', 'used', 'total']
            balance_exchange = balance_exchange.groupby([''])[['free', 'used', 'total']].sum()
    balance_exchange['exchange'] = exc
    for ticker in balance_exchange.index:
        symbol = str(ticker) + '/USDT'
        if ticker == 'USDT':
            balance_exchange.loc[ticker, 'pm_usdt'] = 1
            pass
        elif ticker == 'BRL':
            symbol = 'USDT/BRL'
            brl_price  = exchange.fetchTicker(symbol)['last']
            balance_exchange.loc[ticker, 'pm_usdt'] = 1/brl_price
        elif (ticker == 'BTC') & (exc == 'binance'):
            btc_price = exchange.fetchTicker(symbol)['last']
            balance_exchange.loc[ticker, 'pm_usdt'] = btc_price
        else:
            balance_exchange.loc[ticker, 'pm_usdt'] = exchange.fetchTicker(symbol)['last']

    if len(balance_exchange) > 0:
        balance_exchange['total'] = balance_exchange['total'].astype(float)
        balance_exchange['pm_usdt'] = balance_exchange['pm_usdt'].astype(float)
        balance_exchange = balance_exchange.reset_index(names='coin')
        balance_exchange['total_usd'] = balance_exchange['total']*balance_exchange['pm_usdt']

    balance_exchange_save = pd.concat([balance_exchange_save, balance_exchange])
balance_exchange_save_drop_zero = balance_exchange_save[balance_exchange_save['total_usd'] != 0].copy()

balance_exchange_save_drop_zero['total_brl'] = balance_exchange_save_drop_zero['total_usd']*brl_price
balance_exchange_save_drop_zero['total_btc'] = balance_exchange_save_drop_zero['total_usd']/btc_price
balance_exchange_filt_dust = balance_exchange_save_drop_zero[balance_exchange_save_drop_zero['total_brl'] >= 0.01]
balance_exchange_filt_dust.reset_index(drop=True, inplace=True)

balance_exchange_filt_dust_gm = balance_exchange_filt_dust[(balance_exchange_filt_dust['coin'] == 'USDT') | (balance_exchange_filt_dust['coin'] == 'BRL')].copy()
balance_exchange_filt_dust_gm['type'] = 'liquidity'

balance_exchange_filt_dust_mtm = balance_exchange_filt_dust[~balance_exchange_filt_dust.index.isin(balance_exchange_filt_dust_gm.index)].copy()
balance_exchange_filt_dust_mtm['type'] = 'on_risk'

balance_final = pd.concat([balance_exchange_filt_dust_gm, balance_exchange_filt_dust_mtm]).reset_index(drop=True)
balance_final = balance_final[['coin', 'total', 'exchange', 'total_usd', 'total_brl', 'total_btc', 'type']]
balance_final_red = balance_final[balance_final['total_usd'] > 1].copy()

mexc


NameError: name 'brl_price' is not defined

In [ ]:
print(balance_exchange_filt_dust.groupby(['exchange'])['total_brl'].sum())
print(balance_exchange_filt_dust.groupby(['exchange'])['total_brl'].sum().sum())

exchange
binance             7053.418549
binance_sub         6034.766698
binanceusdm_sub    19609.526531
bybit               3820.254665
mexc                2262.898911
Name: total_brl, dtype: float64
38780.86535362959


In [ ]:
address_btc = ['bc1qmq2anqlqrmkwk56zs3y3yel8d2xp9pk83m8np2', 'bc1qtw5tvmef36p5gtqs7zypra87ujnq8rprns2jtv', 'bc1qx7xug82grxmy6xuhm2n9zjg2mmvtsslx58aqz0', 'bc1qayaneelxwlvmwntuvxpaq76ychalqv2axved9t', 'bc1qz2g8jy8fapsgwp4sh5f20u7hn76r8qt3xsqrgu', 'bc1qa7gkxm3j2ge78nzpc0tm99l2h2pk6f0c7udal7', '3JGnunqzFiZE6onnFqQsirYexfyJCwWQHa']

In [ ]:
mempool = Mempoolpy.MempoolAPI()

info_address = pd.DataFrame()
for address in address_btc:
    infos = pd.DataFrame(mempool.get_address_info(address))
    info_address = pd.concat([info_address, infos])

info_address = info_address.reset_index(names='operation')
info_address['chain_stats'] = info_address['chain_stats'].astype(float)
funded_sum = info_address[info_address['operation'] == 'funded_txo_sum']
btc_on_chain = funded_sum['chain_stats'].sum()
btc_on_chain = btc_on_chain/1e8
filtered_info_address = info_address[info_address['operation'].isin(['funded_txo_sum', 'spent_txo_sum'])]
grouped_sum = filtered_info_address.groupby(['operation'])['chain_stats'].sum()
total_btc = (grouped_sum['funded_txo_sum'] - grouped_sum['spent_txo_sum'])/1e8
total_usd = total_btc*btc_price
vector = [['BTC', total_btc, 'onchain', total_usd, total_usd*brl_price, total_btc, 'on_risk']]
onchain_balance = pd.DataFrame(vector, columns=['coin', 'total', 'exchange', 'total_usd', 'total_brl', 'total_btc', 'type'])
balance_wout_shits = pd.concat([balance_final_red, onchain_balance])
balance_wout_shits.reset_index(drop=True, inplace=True)
balance_wout_shits['date_time'] = date_time
balance_wout_shits

,coin,total,exchange,total_usd,total_brl,total_btc,type,date_time
0,BRL,2089.064541,binance,369.288411,2089.064541,0.005551,liquidity,2024-07-25 23:20:47.814021
1,USDT,776.185935,binance,776.185935,4390.883834,0.011667,liquidity,2024-07-25 23:20:47.814021
2,USDT,400.000013,mexc,400.000013,2262.800073,0.006012,liquidity,2024-07-25 23:20:47.814021
3,USDT,675.297769,bybit,675.297769,3820.159478,0.010150,liquidity,2024-07-25 23:20:47.814021
4,USDT,1066.297732,binance_sub,1066.297732,6032.046273,0.016028,liquidity,2024-07-25 23:20:47.814021
5,USDT,3466.417983,binanceusdm_sub,3466.417983,19609.526531,0.052104,liquidity,2024-07-25 23:20:47.814021
6,BTC,0.001510,binance,100.430198,568.133628,0.001510,on_risk,2024-07-25 23:20:47.814021
7,BTC,0.371572,onchain,24720.317304,139842.834987,0.371572,on_risk,2024-07-25 23:20:47.814021


In [ ]:
balance_wout_shits['date_time'] = date_time

In [ ]:
df = pd.read_excel('out/balance.xlsx', index_col=0)
df = pd.concat([df, balance_wout_shits])
balance_wout_shits.to_excel('out/balance.xlsx')


In [ ]:
balance_wout_shits

,coin,total,exchange,total_usd,total_brl,total_btc,type,date_time
0,BRL,2089.064541,binance,369.288411,2089.064541,0.005551,liquidity,2024-07-25 23:20:47.814021
1,USDT,776.185935,binance,776.185935,4390.883834,0.011667,liquidity,2024-07-25 23:20:47.814021
2,USDT,400.000013,mexc,400.000013,2262.800073,0.006012,liquidity,2024-07-25 23:20:47.814021
3,USDT,675.297769,bybit,675.297769,3820.159478,0.010150,liquidity,2024-07-25 23:20:47.814021
4,USDT,1066.297732,binance_sub,1066.297732,6032.046273,0.016028,liquidity,2024-07-25 23:20:47.814021
5,USDT,3466.417983,binanceusdm_sub,3466.417983,19609.526531,0.052104,liquidity,2024-07-25 23:20:47.814021
6,BTC,0.001510,binance,100.430198,568.133628,0.001510,on_risk,2024-07-25 23:20:47.814021
7,BTC,0.371572,onchain,24720.317304,139842.834987,0.371572,on_risk,2024-07-25 23:20:47.814021


In [ ]:
balance_wout_shits.groupby(['exchange'])[['total_usd', 'total_brl', 'total_btc']].sum()

,total_usd,total_brl,total_btc
exchange,,,
binance,1245.904544,7048.082003,0.018727
binance_sub,1066.297732,6032.046273,0.016028
binanceusdm_sub,3466.417983,19609.526531,0.052104
bybit,675.297769,3820.159478,0.010150
mexc,400.000013,2262.800073,0.006012
onchain,24720.317304,139842.834987,0.371572


In [ ]:
balance_wout_shits.groupby(['exchange'])[['total_usd', 'total_brl', 'total_btc']].sum().sum()

total_usd     31574.235345
total_brl    178615.449346
total_btc         0.474593
dtype: float64

In [ ]:
nft_wallets = ['bc1phm4nmytj7dqknrv0507dzn7czp6tf5tnrhsnklymw6aguptqws6qrt4728']

analise excel

In [ ]:
ordi = Ordipy.OrdiScan()
# get_nft_balance = ordi.get_ordinals_balance(nft_wallets[0])
get_runes_balance = ordi.get_runes_balance(nft_wallets[0])

In [ ]:
get_nft_balance

{'data': [{'inscription_id': 'c5a4225c362fd663c28687bd1c4d0bb0fa7bae17d11803404cbae8b0d09c6890i6',
   'inscription_number': 70752003,
   'content_type': 'image/png',
   'owner_address': 'bc1phm4nmytj7dqknrv0507dzn7czp6tf5tnrhsnklymw6aguptqws6qrt4728',
   'owner_output': '4ecf25897b08b18b06761556778646b98c7050c79bd4ff4e449849d62f92a463:1',
   'genesis_address': 'bc1pqu0saqst4e6cjwf76dvhnwf32eu96z4zr9qehn8ylgd56fxxnn0qj5ecrw',
   'genesis_output': 'c5a4225c362fd663c28687bd1c4d0bb0fa7bae17d11803404cbae8b0d09c6890:6',
   'timestamp': '2024-05-14T04:00:13.000Z',
   'metadata': None,
   'metaprotocol': None,
   'sat': 45015189413,
   'content_url': 'https://ordiscan.com/content/c5a4225c362fd663c28687bd1c4d0bb0fa7bae17d11803404cbae8b0d09c6890i6',
   'parent_inscription_id': 'de132d7f50cb6a0c749ea66e42b2cab1315b05b0ee717dffc442d1d10ec374b2i0',
   'delegate_inscription_id': None,
   'satributes': ['BLOCK_9', 'NAKAMOTO', 'FIRST_TX', 'VINTAGE'],
   'submodules': []},
  {'inscription_id': '1d28822

In [ ]:
df_runes = pd.DataFrame(get_runes_balance['data'])
df_runes.set_index('name', inplace=True)
df_adjust = pd.read_excel('in/rune_adjustment.xlsx', index_col=0)
df_adjust.set_index('name', inplace=True)
df_runes['adjust'] = df_adjust['adjustment']
df_runes['real_balance'] = df_runes['balance'].astype(float)/df_runes['adjust'].astype(float)


In [ ]:
df_runes['real_balance'] = df_runes['balance'].astype(float)/df_runes['adjust'].astype(float)
df_runes

,balance,adjust,real_balance
name,,,
EPICEPICEPICEPIC,2750000,100,27500.00
LIQUIDIUMTOKEN,68189,100,681.89
LOBOTHEWOLFPUP,10393500000000,100000000,103935.00
ZBITBLUEBITCOIN,20000000000,100000000,200.00
PEPEWITHONKERS,800850000,10000,80085.00
UNCOMMONGOODS,1,1,1.00
GOLDRUNESTONE,100000000000,100000000,1000.00
COOKTHEMEMPOOL,5040,1,5040.00
FUCKBITCHESGETBITCOIN,222222,1,222222.00


In [ ]:
df_rune_token_save = pd.DataFrame()
for token in df_runes.index:
    rune_token = ordi.get_rune_info(token)
    df_rune_token = pd.DataFrame([rune_token['data']])
    df_rune_token['name'] = token
    df_rune_token_save = pd.concat([df_rune_token_save, df_rune_token])
df_rune_token_save


,price_in_sats,price_in_usd,market_cap_in_btc,market_cap_in_usd,name
0,13.000000,0.008705,130.013,8705670,EPICEPICEPICEPIC
0,515.000000,0.344844,515.000,34484400,LIQUIDIUMTOKEN
0,1.550002,0.001038,325.500,21795480,LOBOTHEWOLFPUP
0,999.000000,0.668930,209.790,14047538,ZBITBLUEBITCOIN
0,0.063000,0.000042,4.010,268510,PEPEWITHONKERS
0,468.000000,0.313373,58.474,3915419,UNCOMMONGOODS
0,0.000000,0.000000,0.000,0,GOLDRUNESTONE
0,3.479443,0.002330,31.521,2110646,COOKTHEMEMPOOL
0,0.000000,0.000000,0.000,0,FUCKBITCHESGETBITCOIN
0,1458.000000,0.976277,308.679,20669146,SATOSHINAKAMOTO


In [ ]:
df_rune_token_save.set_index('name', inplace=True)

In [ ]:
df_runes

,balance,adjust,real_balance
name,,,
EPICEPICEPICEPIC,2750000,100,27500.00
LIQUIDIUMTOKEN,68189,100,681.89
LOBOTHEWOLFPUP,10393500000000,100000000,103935.00
ZBITBLUEBITCOIN,20000000000,100000000,200.00
PEPEWITHONKERS,800850000,10000,80085.00
UNCOMMONGOODS,1,1,1.00
GOLDRUNESTONE,100000000000,100000000,1000.00
COOKTHEMEMPOOL,5040,1,5040.00
FUCKBITCHESGETBITCOIN,222222,1,222222.00


In [ ]:
df_rune_token_save

,price_in_sats,price_in_usd,market_cap_in_btc,market_cap_in_usd
name,,,,
EPICEPICEPICEPIC,13.000000,0.008705,130.013,8705670
LIQUIDIUMTOKEN,515.000000,0.344844,515.000,34484400
LOBOTHEWOLFPUP,1.550002,0.001038,325.500,21795480
ZBITBLUEBITCOIN,999.000000,0.668930,209.790,14047538
PEPEWITHONKERS,0.063000,0.000042,4.010,268510
UNCOMMONGOODS,468.000000,0.313373,58.474,3915419
GOLDRUNESTONE,0.000000,0.000000,0.000,0
COOKTHEMEMPOOL,3.479443,0.002330,31.521,2110646
FUCKBITCHESGETBITCOIN,0.000000,0.000000,0.000,0


In [ ]:
df_runes_balance = pd.concat([df_rune_token_save, df_runes], axis=1)
df_runes_balance['total_sats'] = df_runes_balance['price_in_sats']*df_runes_balance['real_balance']
df_runes_balance['total_usd'] = df_runes_balance['price_in_usd']*df_runes_balance['real_balance']
df_runes_balance['total_btc'] = df_runes_balance['total_sats']/1e8
df_runes_balance['']

,price_in_sats,price_in_usd,market_cap_in_btc,market_cap_in_usd,balance,adjust,real_balance,total_sats,total_usd,total_btc
name,,,,,,,,,,
EPICEPICEPICEPIC,13.000000,0.008705,130.013,8705670,2750000,100,27500.00,357500.000000,239.387500,0.003575
LIQUIDIUMTOKEN,515.000000,0.344844,515.000,34484400,68189,100,681.89,351173.350000,235.145675,0.003512
LOBOTHEWOLFPUP,1.550002,0.001038,325.500,21795480,10393500000000,100000000,103935.00,161099.503844,107.884530,0.001611
ZBITBLUEBITCOIN,999.000000,0.668930,209.790,14047538,20000000000,100000000,200.00,199800.000000,133.786000,0.001998
PEPEWITHONKERS,0.063000,0.000042,4.010,268510,800850000,10000,80085.00,5045.355000,3.363570,0.000050
UNCOMMONGOODS,468.000000,0.313373,58.474,3915419,1,1,1.00,468.000000,0.313373,0.000005
GOLDRUNESTONE,0.000000,0.000000,0.000,0,100000000000,100000000,1000.00,0.000000,0.000000,0.000000
COOKTHEMEMPOOL,3.479443,0.002330,31.521,2110646,5040,1,5040.00,17536.390244,11.743200,0.000175
FUCKBITCHESGETBITCOIN,0.000000,0.000000,0.000,0,222222,1,222222.00,0.000000,0.000000,0.000000


In [ ]:
binance_raw = pd.read_csv('in/all_op_binance.csv')

Analise Saques

gerando analise depositos/saque em BRL

In [ ]:
# binance_deposit = binance_raw_start[binance_raw_start['Operation'] == 'Deposit'].reset_index(drop=True)
# binance_deposit_brl = binance_deposit[binance_deposit['Coin'] == 'BRL'].reset_index(drop=True)
# binance_with_fiat = binance_raw_start[binance_raw_start['Operation'] == 'Fiat Withdraw'].reset_index(drop=True)
# binance_with_fiat_brl = binance_with_fiat[binance_with_fiat['Coin'] == 'BRL'].reset_index(drop=True)
# binance_real = pd.concat([binance_with_fiat_brl, binance_deposit_brl])
# binance_real = binance_real.sort_values(by='UTC_Time')
# binance_real['BRL_CUMSUM'] = binance_real['Change'].cumsum()
# binance_real

testes

In [ ]:
op_futures = ['Transfer Between Spot Account and UM Futures Account', 'Fee',
       'Funding Fee', 'Realized Profit and Loss', 'Delisting Settlement',
       'Insurance Fund Compensation',
       'Transfer Between Main Account/Futures and Margin Account']

In [ ]:
binance_raw_start = binance_raw[['UTC_Time', 'Account', 'Operation', 'Coin', 'Change']].copy()
binance_raw_start['UTC_Time'] = pd.to_datetime(binance_raw_start['UTC_Time'])

op_futures = ['Transfer Between Spot Account and UM Futures Account', 'Fee',
       'Funding Fee', 'Realized Profit and Loss', 'Delisting Settlement',
       'Insurance Fund Compensation',
       'Transfer Between Main Account/Futures and Margin Account']

binance_raw_start_filt1 = binance_raw_start[binance_raw_start['Operation'] != op_futures[6]].copy()

# fase 1
binance_raw_start_filt1['Coin_usd'] = binance_raw_start_filt1['Coin'].replace('BUSD', 'USD').replace('USDT', 'USD')
binance_raw_start_filt1['Coin_usd'] = binance_raw_start_filt1['Coin_usd'].replace('USDT', 'USD')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation'].replace('Transfer Between Spot Account and UM Futures Account', 'Management')

binance_raw_start_filt1['Account_usd'] = binance_raw_start_filt1['Account'].replace('USD-M Futures', 'Futures')

#fase 2
binance_raw_start_filt1['Account_usd'] = binance_raw_start_filt1['Account_usd'].replace('Isolated Margin', 'Spot')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Transaction Buy', 'm1')
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Transaction Fee', 'Fee')
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Transaction Spend', 'm2')

binance_raw_start_filt1['Account_usd'] = binance_raw_start_filt1['Account_usd'].replace('Funding', 'Spot')
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Launchpool Earnings Withdrawal', 'Earnings')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Launchpool Subscription/Redemption', 'Management')

#não verificados

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Simple Earn Flexible Subscription', 'Management') #verificar flexible
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Simple Earn Flexible Redemption', 'Management')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Simple Earn Flexible Interest', 'Earnings')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Commission History', 'Earnings')


binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Transaction Revenue', 'Buy')
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Transaction Sold', 'Sell')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Airdrop Assets', 'Earnings')

binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Insurance Fund Compensation', 'Other')
binance_raw_start_filt1['Operation_usd'] = binance_raw_start_filt1['Operation_usd'].replace('Delisting Settlement', 'Other')

ops = ['m1', 'm2', 'Small Assets Exchange BNB', 'Binance Convert']
binance_raw_start_filt2 = binance_raw_start_filt1[~binance_raw_start_filt1['Operation_usd'].isin(ops)].copy()

binance_raw_start_filt1_ = binance_raw_start_filt1[binance_raw_start_filt1['Operation_usd'].isin(ops)].copy()

binance_raw_start_filt1_['Operation_usd'] = binance_raw_start_filt1_.apply(lambda x: 'Buy' if x['Change'] > 0 else 'Sell', axis=1)

binance_raw_start_filt3 = pd.concat([binance_raw_start_filt2, binance_raw_start_filt1_])
binance_raw_start_filt3 = binance_raw_start_filt3.sort_values(by='UTC_Time')
binance_raw_start_filt3.to_excel('out/analytics_binance.xlsx')

In [ ]:
binance_raw_start.groupby(['Coin', 'Operation'])[['Change']].sum().reset_index()


,Coin,Operation,Change
0,1000SATS,Transaction Buy,1145.00000
1,1000SATS,Transaction Fee,-1144.82000
2,1000SATS,Transfer Between Main Account/Futures and Marg...,0.00000
3,ADA,Buy,543.62000
4,ADA,Fee,-0.21000
...,...,...,...
116,XTZ,Simple Earn Flexible Subscription,-47.13130
117,XTZ,Small Assets Exchange BNB,-0.00971
118,XTZ,Transaction Buy,47.10000
119,XTZ,Transaction Sold,-167.10000


pegar todos os mercados

In [ ]:
markets = pd.DataFrame(exchange.loadMarkets()).loc[["id"]].T.reset_index(drop=True) #filtrando mercados
# new_market = markets[markets.index > 2942].drop_duplicates()

loop para pegar todas operações

In [ ]:
# k = 0
# df_my_trades_ticker = pd.DataFrame(columns=['timestamp', 'datetime', 'symbol', 'id', 'order', 'type', 'side',
#        'takerOrMaker', 'price', 'amount', 'cost', 'cost', 'currency'])
# for ticker in markets['id'].unique():
#     print(ticker)
#     flag_finalizado = False
#     tempo_inicial = None
#     df_my_trades = pd.DataFrame(columns=['info', 'timestamp', 'datetime', 'symbol', 'id', 'order', 'type',
#         'side', 'takerOrMaker', 'price', 'amount', 'cost', 'fee', 'fees'])
#     while flag_finalizado is False:
#         print(k)
#         k += 1
#         MyTrades = pd.DataFrame(exchange.fetch_my_trades(symbol=ticker, since=tempo_inicial, limit=None, params={}))
#         if len(MyTrades) > 0:
#             tempo_inicial_loop = MyTrades['timestamp'].iloc[-1]
#             if tempo_inicial == tempo_inicial_loop:
#                 flag_finalizado = True
#             else:
#                 df_my_trades = pd.concat([df_my_trades, MyTrades])
#                 tempo_inicial = tempo_inicial_loop
#         else:
#             flag_finalizado = True
#     if len(df_my_trades) > 0:
#         df_my_trades.reset_index(inplace=True, drop=True)
#         df_my_trades = pd.concat([df_my_trades, pd.json_normalize(df_my_trades['fee'])], axis=1)
#         df_my_trades.drop(columns=['info', 'fee', 'fees'], inplace=True)
#         df_my_trades.drop_duplicates(inplace=True)
#         df_my_trades_ticker = pd.concat([df_my_trades_ticker, df_my_trades])

# df_my_trades_ticker.columns.values[11] = 'cost_fee'
# df_my_trades_ticker['datetime'] = pd.to_datetime(df_my_trades_ticker['datetime'])
# df_my_trades_ticker['datetime'] = df_my_trades_ticker['datetime'].dt.tz_localize(None)
# df_my_trades_ticker.to_excel('BINANCEUSDM.xlsx', index=False)

fetch balance

In [ ]:
balance = pd.DataFrame(exchange.fetchBalance(params={}))
balance_to_pivot = balance.drop(columns=['info'])
balanceNow = balance_to_pivot[['free', 'used', 'total']].dropna()
balance_exchange = balanceNow[balanceNow['total'] != 0]
balance_exchange['net_balance'] = exchange
balance_exchange

C:\Users\Felipe\AppData\Local\Temp\ipykernel_1616\3601976719.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_exchange['net_balance'] = exchange


,free,used,total,net_balance
USDT,1371.246372,2088.844689,3460.188134,Binance USDⓈ-M


read op

In [ ]:
raw_op = pd.read_excel('raw_op.xlsx')
raw_op.drop(columns=['type'], inplace=True)
raw_op['market'] = raw_op['symbol'].str.replace('/', '')
# raw_op = pd.concat([raw_op, raw_op['symbol'].str.split('/', expand=True)], axis=1)
currency = ['BTC', 'USD', 'BRL']
raw_op['symbol_usd'] = raw_op['symbol'].str.replace('USDT', 'USD').str.replace('BUSD', 'USD')
raw_op['currency_usd'] = raw_op['currency'].str.replace('USDT', 'USD').str.replace('BUSD', 'USD')
raw_op = pd.concat([raw_op, raw_op['symbol_usd'].str.split('/', expand=True)], axis=1)

raw_op['fee_usd'] = raw_op.apply(lambda x: x['price'] * x['cost_fee'] if (x['currency_usd'] != 'USD') and (x['symbol'][1] == 'USD') else x['cost_fee'], axis=1)
raw_op['bet'] = raw_op.apply(lambda x: x['price'] * x['amount'] if x['side']=='buy' else -x['price'] * x['amount'], axis=1)
raw_op['net_amount'] = raw_op.apply(lambda x: x['amount'] if x['side']=='buy' else -x['amount'], axis=1)
raw_op['fee_usd'] = -raw_op['fee_usd']
raw_op['symbol'].str.split('/', expand=True)
raw_op.groupby(['symbol_usd', 'side'])[['bet', 'net_amount']].sum().reset_index()

FileNotFoundError: [Errno 2] No such file or directory: 'raw_op.xlsx'

In [ ]:
all_coins_had = raw_op['symbol'].str.split('/', expand=True).stack().reset_index(drop=True).unique()
all_coins_had

array(['BTC', 'USDT', 'BNB', 'ADA', 'BUSD', 'XTZ', 'OGN', 'DOT', 'CAKE',
       'BRL', 'ETH', 'FLOKI', 'WLD', 'ORDI'], dtype=object)

In [ ]:
# all_coins_had = raw_op['symbol'].str.split('/', expand=True).stack().reset_index(drop=True).unique()

# sample = [[k1, k2]
#           for k1 in all_coins_had
#           for k2 in range(50)]

# deposits_save = pd.DataFrame()
# for k in range(50):
#     k1 = k*86400000  + 1655000000000
#     print([k1])
#     deposits = pd.DataFrame(exchange.fetchDeposits(since=k1))
#     if len(deposits) > 0:
#         deposits_save = pd.concat([deposits_save, deposits])
# deposits_save

[1655000000000]
[1655086400000]
[1655172800000]
[1655259200000]
[1655345600000]
[1655432000000]
[1655518400000]
[1655604800000]
[1655691200000]
[1655777600000]
[1655864000000]
[1655950400000]
[1656036800000]
[1656123200000]
[1656209600000]
[1656296000000]
[1656382400000]
[1656468800000]
[1656555200000]
[1656641600000]
[1656728000000]
[1656814400000]
[1656900800000]
[1656987200000]
[1657073600000]
[1657160000000]
[1657246400000]
[1657332800000]
[1657419200000]
[1657505600000]
[1657592000000]
[1657678400000]
[1657764800000]
[1657851200000]
[1657937600000]
[1658024000000]
[1658110400000]
[1658196800000]
[1658283200000]
[1658369600000]
[1658456000000]
[1658542400000]
[1658628800000]
[1658715200000]
[1658801600000]
[1658888000000]
[1658974400000]
[1659060800000]
[1659147200000]
[1659233600000]


""


In [ ]:
exchange.fetchDeposits(code='BRL')

[{'info': {'orderNo': '03506158189218541568070647',
   'fiatCurrency': 'BRL',
   'indicatedAmount': '5000.00',
   'amount': '5000.00',
   'totalFee': '0',
   'method': 'Bank Transfer (PIX)',
   'status': 'Successful',
   'createTime': '1720291265000',
   'updateTime': '1720291365000',
   'type': 'deposit'},
  'id': '03506158189218541568070647',
  'txid': None,
  'timestamp': 1720291265000,
  'datetime': '2024-07-06T18:41:05.000Z',
  'network': None,
  'address': None,
  'addressTo': None,
  'addressFrom': None,
  'tag': None,
  'tagTo': None,
  'tagFrom': None,
  'type': 'deposit',
  'amount': 5000.0,
  'currency': 'BRL',
  'status': 'ok',
  'updated': 1720291365000,
  'internal': None,
  'comment': None,
  'fee': {'currency': 'BRL', 'cost': 0.0}},
 {'info': {'orderNo': '03506163771031853056070647',
   'fiatCurrency': 'BRL',
   'indicatedAmount': '5000.00',
   'amount': '5000.00',
   'totalFee': '0',
   'method': 'Bank Transfer (PIX)',
   'status': 'Successful',
   'createTime': '17202

In [ ]:
# a = 0
# for k in range(1000):
#     print(k)
#     k1 = 1514772000000 + 1555244000*k
#     k2 = k1 + 1555244000
#     response = exchange.sapiGetAssetAssetDividend({"startTime":
#         k1, "endTime": k2, "limit": 500})
#     r1 = pd.DataFrame(response)
#     if len(r1) > 0:
#         if a == 0:
#             r2 = pd.DataFrame(columns = r1.columns)
#             r2 = pd.concat([r2, r1])
#             a = a + 1
#         else:
#             r2 = pd.concat([r2, r1])

# a = r2.rows.apply(lambda x: x['id'])
# a.drop_duplicates()

In [ ]:
# a = 0
# for k in range(150):
#     print(k)
#     k1 = 13*k*864000000  + 1514772000000
#     df1 = exchange.fetchWithdrawals(since=k1) #1 jan 2018
#     df2 = pd.DataFrame(df1)
#     if a == 0:
#         df3 = pd.DataFrame(columns=df2.columns)
#         df3 = pd.concat([df3, df2])
#         a = a + 1
#     else:
#         df3 = pd.concat([df3, df2])
# df3


# # a = 0
# # kx = []     
# # for k in range(50):
# #     print(k)
# #     k1 = k*86400000*153  + 1655000000000
# #     df1 = exchange.fetchTransfers(since = k1) #1 jan 2018
# #     df2 = pd.DataFrame(df1)
# #     if a == 0:
# #         df4 = pd.DataFrame( columns = df2.columns)
# #         df4 = pd.concat([df4, df2])
# #         a = a + 1
# #     else:
# #         df4 = pd.concat([df4, df2])
# #     if len(df2) > 0:
# #         kx.append(k1)

In [ ]:
exchange.has

{'publicAPI': True,
 'privateAPI': True,
 'CORS': None,
 'spot': True,
 'margin': True,
 'swap': True,
 'future': True,
 'option': True,
 'addMargin': True,
 'cancelAllOrders': True,
 'cancelOrder': True,
 'cancelOrders': True,
 'createDepositAddress': False,
 'createLimitOrder': True,
 'createMarketOrder': True,
 'createOrder': True,
 'createPostOnlyOrder': True,
 'createReduceOnlyOrder': True,
 'createStopOrder': True,
 'createStopLimitOrder': True,
 'createStopMarketOrder': False,
 'editOrder': True,
 'fetchAccounts': None,
 'fetchBalance': True,
 'fetchBidsAsks': True,
 'fetchBorrowInterest': True,
 'fetchBorrowRate': None,
 'fetchBorrowRateHistory': True,
 'fetchBorrowRatesPerSymbol': None,
 'fetchBorrowRates': None,
 'fetchCanceledOrders': 'emulated',
 'fetchClosedOrder': False,
 'fetchClosedOrders': 'emulated',
 'fetchClosedOrdersWs': None,
 'fetchConvertCurrencies': True,
 'fetchConvertQuote': True,
 'fetchConvertTrade': True,
 'fetchConvertTradeHistory': True,
 'fetchCrossBorr